In [ ]:
# Ejecutar esta celda solo si la celda de abajo tuvo problemas
# Recargar página luego de ejecutar esta celda por primera vez
!pip install "ipywidgets>=7.2"
!pip install sklearn
!pip install tensorflow
!pip install pandas
!pip install numpy
!pip install bqplot
!jupyter nbextension enable --py --sys-prefix bqplot

In [ ]:
import pandas as pd
import numpy as np
import bqplot.pyplot as plt
import tensorflow as tf
from IPython.display import display
import ipywidgets as widgets

from bqplot import (ColorScale, DateColorScale, OrdinalColorScale, 
                    LinearScale, Tooltip)
from ipywidgets import interact, interactive, fixed, interact_manual

# Evaluación de modelos

In [ ]:
# Importar las funciones de los modelos
from TimeSeriesPrediction import fixed_partitioning_predict
from LinearRegression import linear_regression_predict
from DNN import load_model, dnn_predict, dnn_predict_by_year

# Importar interface de modelos
from Evaluation import Model, TestResult

# Crear modelos

model_dict = {
    'Fix. part. time series' : Model(fixed_partitioning_predict),
    'Fix. part. linear regression' : Model(linear_regression_predict),
    'Deep Neural Networks' : Model(dnn_predict, args = dict(cached_model = load_model('default')))
}

# Crear etiquetas
label_dict = {
    'Fix. part. time series' : 'Fixed partitioning time series',
    'Fix. part. linear regression' : 'Fixed partitioning linear regression',
    'Deep Neural Networks' : 'Redes neuronales', 
    'Primarias públicas y privadas' : 'PrimariasCompletas',
    'Primarias públicas' : 'PrimariasPublicas',
    'Primarias privadas' : 'PrimariasPrivadas'
}

In [ ]:
# Seleccionar modelo de predicción
model_radio_button = widgets.RadioButtons(
    options=['Fix. part. time series', 'Fix. part. linear regression', 'Deep Neural Networks'],
    description='Modelo',
    disabled=False
)

display(model_radio_button)

In [ ]:
# Seleccionar conjunto de datos
set_name_radio_button = widgets.RadioButtons(
    options=['Primarias públicas y privadas', 'Primarias públicas', 'Primarias privadas'],
    description='Conjunto',
    disabled=False
)

display(set_name_radio_button)

In [ ]:
# Seleccionar años a predecir
prediction_size_int_slider = widgets.IntSlider(
    value=3,
    min=1,
    max=5,
    step=1,
    description='Años',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

display(prediction_size_int_slider)

In [ ]:
# Calcular resultados de la prueba
model = model_dict[model_radio_button.value]
result = model.test_set(label_dict[set_name_radio_button.value], prediction_size_int_slider.value)

n = int(result.Y.shape[0] / prediction_size_int_slider.value)
m = max(np.amax(result.Y_hat), np.amax(result.Y))

colors = []
if prediction_size_int_slider.value >= 1:
    colors.append(1)
if prediction_size_int_slider.value >= 2:
    colors.append(2)
if prediction_size_int_slider.value >= 3:
    colors.append(3)
if prediction_size_int_slider.value >= 4:
    colors.append(4)
if prediction_size_int_slider.value >= 5:
    colors.append(5)

colors = np.array(colors * n)

fig = plt.figure(title = 'Predicción de matrícula escolar en %s utilizando %s' % (set_name_radio_button.value,model_radio_button.value), legend_location='top-left', fig_margin=dict(top=50, bottom=70, left=100, right=100))
plt.scales(scales={'color': OrdinalColorScale(colors=['Green', 'DodgerBlue', 'Yellow', 'Orange', 'Red'])})

axes_options = {'x': dict(label='Valor real (alumnos)'),
                'y': dict(label='Valor predecido (alumnos)'),
                'color': dict(label='Año', side='right')
               }

scatter2 = plt.scatter(result.Y,
                       result.Y_hat,
                       color=colors,
                       stroke='black',
                       axes_options=axes_options)
plt.plot(x = np.array([0, m]), y = np.array([0, m]), labels = ['Línea base de predicción'], display_legend = True)

fig

# Predicción de matrícula en una sola escuela

In [ ]:
out = widgets.Output()

@interact(cct = 'Introduce cct', 
          anios = widgets.IntSlider(min=1, max=5, step=1, value=3, description = 'Años'), 
          modelo = widgets.RadioButtons(
              options=['Fix. part. time series', 
                       'Fix. part. linear regression', 
                       'Deep Neural Networks'],
              description='Modelo',
              disabled=False
))
def prediccion(cct, anios, modelo) :
    global model_dict
    model = model_dict[modelo]
    
    dataset = pd.read_csv('PrimariasCompletas.csv')
    unique_index = pd.Index(list(dataset['cct']))
    if cct in unique_index :
        index = unique_index.get_loc(cct)
        print('Se encontró el cct')
    else :
        print('No se encontró el cct')
        return
    
    # Quitar el cct
    row = np.array(dataset.loc[index][1:])
    
    # Separamos los últimos 5 años
    X = row[: -model.TEST_SIZE]

    if anios == model.TEST_SIZE :
        Y = row[-model.TEST_SIZE :]
    else :
        Y = row[-model.TEST_SIZE : -(model.TEST_SIZE - anios)]
    prediccion = model.predict(X, anios)
    
    list_X = list(X)
    list_Y = list(Y)
    list_predict = list(prediccion)
    
    with out:
        out.clear_output(True)
        
        axes_options = {'x': {'label': 'Año'},'y': {'label': 'Alumnos'}}
        
        fig = plt.figure(title = 'Predicción de matrícula escolar en la escuela %s utilizando %s' % (cct, modelo), legend_location='top-left')
        plt.plot(x = [1998 + i for i in range(len(X))], y = list_X, colors = ['red'], axes_options=axes_options, labels = ['Datos de entrenamiento'], display_legend = True, marker = 'circle')
        plt.plot(x = [1997 + len(X) + i  for i in range(len(Y) + 1)], y = (list_X + list_Y)[-(anios + 1):], labels = ['Datos de prueba'], display_legend = True, marker = 'circle')
        plt.plot(x = [1997 + len(X) + i  for i in range(len(Y) + 1)], y = (list_X + list_predict)[-(anios + 1):], colors = ['green'], labels = ['Datos predecidos'], display_legend = True, marker = 'circle')
        
        plt.show()
out